Reconstructing a lineage tree from mutation table

In [1]:
import os
import sys
sys.path.append('path_to_examples')

from utils import run_command

import pandas as pd

In [2]:
name = 'SubC3_train_000X'

In [3]:
project_dir = 'path_to_Subchallenge3/{}'.format(name)


Evaluating the tree reconstruction against a know reference using TreeCMP

In [4]:
def compare(path_simulation_newick, path_reconstructed_newick, path_score_output):

    path_treecmp = 'path_to_treecmp/bin/TreeCmp/bin/treeCmp.jar'
    
    #  Metrics for rooted trees
    metrics = {
        "rooted": "rc rf tt",
        "unrooted": "ms rf pd qt um"
    }

    cmd = [
        'java', '-jar', path_treecmp,
        '-P', '-N', '-I'
        '-r', path_simulation_newick,
        '-i', path_reconstructed_newick,
        '-o', path_score_output,
        '-d'
    ]

    cmd.extend(metrics['rooted'].split(' '))

    run_command(cmd)
    
    
def get_scores(path_simulation_newick, path_reconstructed_newick, path_score_output):

    compare(path_simulation_newick, path_reconstructed_newick, path_score_output)

    df_metrics = pd.read_csv(
        path_score_output,
        sep='\t',
        nrows=1
    )

    return df_metrics

In [5]:
scores = get_scores(
    os.path.join(project_dir,os.path.join(project_dir,'{}_reference_tree.newick'.format(name))),
    os.path.join(project_dir,os.path.join(project_dir,'{}_reconstructed_tree.newick'.format(name))),
    '/tmp/scores.txt'
)

In [6]:
scores.T

,0
No,1.000000e+00
RefTree,1.000000e+00
Tree,1.000000e+00
RefTree_taxa,5.810000e+02
Tree_taxa,5.820000e+02
Common_taxa,5.810000e+02
R-F_Cluster,1.915000e+02
R-F_Cluster_toYuleAvg,3.309000e-01
R-F_Cluster_toUnifAvg,3.308000e-01
R-F,1.910000e+02
